# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import optuna
import pathlib
import os
from omegaconf import OmegaConf

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*exists and is not empty.*")
warnings.filterwarnings("ignore", ".*is smaller than the logging interval Trainer.*")


# Prepaer data for legacy training

## Load immunomarker regression data

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)' # 'origin' 'imp_source(imm)_method(knn)_params(5)' 'imp_source(imm)_method(miceforest)_params(2)'

selection_method = 'mrmr' # 'f_regression' 'spearman' 'mrmr'
n_feats = 100

imm = 'CXCL9'

tst_n_splits = 5
tst_n_repeats = 5
tst_random_state = 1337
tst_split_id = 5

val_n_splits = 4
val_n_repeats = 2
val_random_state = 1337
val_fold_id = 5

fn_samples = f"samples_tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats})"
with open(f"D:/YandexDisk/Work/bbd/immunology/003_EpImAge/{fn_samples}.pickle", 'rb') as handle:
    samples = pickle.load(handle)
    
for split_id in range(tst_n_splits * tst_n_repeats):
    for fold_id in range(val_n_splits * val_n_repeats):
        test_samples = samples[split_id]['test']
        train_samples = samples[split_id]['trains'][fold_id]
        validation_samples = samples[split_id]['validations'][fold_id]

        intxns = {
            'train_validation': set.intersection(set(train_samples), set(validation_samples)),
            'validation_test': set.intersection(set(validation_samples), set(test_samples)),
            'train_test': set.intersection(set(train_samples), set(test_samples))
        }
        
        for intxn_name, intxn_samples in intxns.items():
            if len(intxn_samples) > 0:
                print(f"Non-zero {intxn_name} intersection ({len(intxn_samples)}) for {split_id} Split and {fold_id} Fold!")

path_data = f"D:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/{imm}"
pathlib.Path(f"{path_data}/pytorch_tabular").mkdir(parents=True, exist_ok=True)
path_configs = "D:/Work/bbs/notebooks/immunology/003_EpImAge/immuno_regression_configs"
data = pd.read_excel(f"{path_data}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path_data}/feats_con.xlsx", index_col=0).index.values.tolist()

split_dict = samples[tst_split_id]

test = data.loc[split_dict['test'], feats + [f"{imm}_log"]]
train = data.loc[split_dict['trains'][val_fold_id], feats + [f"{imm}_log"]]
validation = data.loc[split_dict['validations'][val_fold_id], feats + [f"{imm}_log"]]

## Load age regression data

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)' # 'origin' 'imp_source(imm)_method(knn)_params(5)' 'imp_source(imm)_method(miceforest)_params(2)'

selection_method = 'mrmr' # 'f_regression' 'spearman' 'mrmr'
n_feats = 100

tst_n_splits = 5
tst_n_repeats = 5
tst_random_state = 1337
tst_split_id = 20

val_n_splits = 4
val_n_repeats = 4
val_random_state = 1337
val_fold_id = 5

path_data = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/EpImAge"


data = pd.read_excel(f"{path_data}/data_filtered.xlsx", index_col=0)
data = data.loc[data['Status'] == 'Control', :]
data['Index'] = data.index.values

feats = pd.read_excel(f"{path_data}/feats.xlsx", index_col=0).index.values.tolist()
feats = [f"{f}_log" for f in feats]

with open(f"{path_data}/samples_tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats}).pickle", 'rb') as handle:
    samples = pickle.load(handle)
for split_id in range(tst_n_splits * tst_n_repeats):
    for fold_id in range(val_n_splits * val_n_repeats):
        test_samples = samples[split_id]['test']
        train_samples = samples[split_id]['trains'][fold_id]
        validation_samples = samples[split_id]['validations'][fold_id]
        intxns = {
            'train_validation': set.intersection(set(train_samples), set(validation_samples)),
            'validation_test': set.intersection(set(validation_samples), set(test_samples)),
            'train_test': set.intersection(set(train_samples), set(test_samples))
        }
        for intxn_name, intxn_samples in intxns.items():
            if len(intxn_samples) > 0:
                print(f"Non-zero {intxn_name} intersection ({len(intxn_samples)}) for {split_id} Split and {fold_id} Fold!")

split_dict = samples[tst_split_id]

test = data.loc[split_dict['test'], feats + ["Age"]]
train = data.loc[split_dict['trains'][val_fold_id], feats + ["Age"]]
validation = data.loc[split_dict['validations'][val_fold_id], feats + ["Age"]]

## Add train, validation, test info in data

In [ ]:
data.loc[split_dict['test'], 'Split'] = 'tst'
data.loc[split_dict['trains'][val_fold_id], 'Split'] = 'trn'
data.loc[split_dict['validations'][val_fold_id], 'Split'] = 'val'

In [ ]:
data.to_excel(f"{path_data}/data_legacy.xlsx")
data.to_pickle(f"{path_data}/data_legacy.pkl")

In [ ]:
df_feats = pd.DataFrame(index=feats)
df_feats.to_excel((f"{path_data}/feats_con_legacy.xlsx"), index_label='Feats')
df_feats.to_pickle((f"{path_data}/feats_con_legacy.pkl"))

## Collect legacy results: immunomarkers

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)' # 'origin' 'imp_source(imm)_method(knn)_params(5)' 'imp_source(imm)_method(miceforest)_params(2)'

selection_method = 'mrmr' # 'f_regression' 'spearman' 'mrmr'
n_feats = 100

imm = 'CXCL9'

model = 'elastic_net'

path_runs = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/{imm}/models/{model}_trn_val_tst/multiruns"

files = glob(f"{path_runs}/*/*/metrics_all_best_*.xlsx")

df_tmp = pd.read_excel(files[0], index_col="metric")
head, tail = os.path.split(files[0])
cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
params = []
for param_pair in cfg:
    param, val = param_pair.split('=')
    params.append(param)
df_res = pd.DataFrame(index=files)

for file in files:

    head, tail = os.path.split(file)
    
    df_res.at[file, 'index'] = head.replace(path_runs, '')
    # Metrics
    df_metrics = pd.read_excel(file, index_col="metric")
    for metric in df_metrics.index.values:
        df_res.at[file, metric + "_val"] = df_metrics.at[metric, "val"]
        df_res.at[file, metric + "_trn"] = df_metrics.at[metric, "trn"]
        df_res.at[file, metric + "_tst"] = df_metrics.at[metric, "tst"]
        df_res.at[file, metric + "_trn_val"] = df_metrics.at[metric, "trn_val"]
        df_res.at[file, metric + "_val_tst"] = df_metrics.at[metric, "val_tst"]
        df_res.at[file, metric + "_trn_val_tst"] = df_metrics.at[metric, "trn_val_tst"]

    # Params
    cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
    for param_pair in cfg:
        param, val = param_pair.split('=')
        df_res.at[file, param] = val

df_res.set_index('index', inplace=True)
df_res["train_more_val"] = False
df_res["selected"] = False
df_res.loc[df_res["mean_absolute_error_trn"] > df_res["mean_absolute_error_val"], "train_more_val"] = True

first_columns = {
    'selected': 'selected',
    'train_more_val': 'train_more_val',
    'mean_absolute_error_trn': 'MAE trn',
    'mean_absolute_error_val': 'MAE val',
    'mean_absolute_error_tst': 'MAE tst',
    'mean_absolute_error_val_tst': 'MAE val_tst',
    'mean_absolute_error_trn_val_tst': 'MAE trn_val_tst',
    'pearson_corr_coef_trn': 'Pcorr trn',
    'pearson_corr_coef_val': 'Pcorr val',
    'pearson_corr_coef_tst': 'Pcorr tst',
    'pearson_corr_coef_val_tst': 'Pcorr val_tst',
    'pearson_corr_coef_trn_val_tst': 'Pcorr trn_val_tst',
    'mean_absolute_error_cv_mean_trn': 'MAE cv_mean_trn',
    'mean_absolute_error_cv_std_trn': 'MAE cv_std_trn',
    'mean_absolute_error_cv_mean_val': 'MAE cv_mean_val',
    'mean_absolute_error_cv_std_val': 'MAE cv_std_val',
    'pearson_corr_coef_cv_mean_trn': 'Pcorr cv_mean_trn',
    'pearson_corr_coef_cv_std_trn': 'Pcorr cv_std_trn',
    'pearson_corr_coef_cv_mean_val': 'Pcorr cv_mean_val',
    'pearson_corr_coef_cv_std_val': 'Pcorr cv_std_val',
}
df_res = df_res[list(first_columns.keys()) + [col for col in df_res.columns if col not in first_columns]]
df_res.rename(columns=first_columns, inplace=True)
df_res.to_excel(f"{path_runs}/summary.xlsx", index=True, index_label="file")

## Collect legacy results: age

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)' # 'origin' 'imp_source(imm)_method(knn)_params(5)' 'imp_source(imm)_method(miceforest)_params(2)'

selection_method = 'mrmr' # 'f_regression' 'spearman' 'mrmr'
n_feats = 100


model = 'elastic_net'

path_runs = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/EpImAge/models/{model}_trn_val_tst/multiruns"

files = glob(f"{path_runs}/*/*/metrics_all_best_*.xlsx")

df_tmp = pd.read_excel(files[0], index_col="metric")
head, tail = os.path.split(files[0])
cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
params = []
for param_pair in cfg:
    param, val = param_pair.split('=')
    params.append(param)
df_res = pd.DataFrame(index=files)

for file in files:

    head, tail = os.path.split(file)
    
    df_res.at[file, 'index'] = head.replace(path_runs, '')
    # Metrics
    df_metrics = pd.read_excel(file, index_col="metric")
    for metric in df_metrics.index.values:
        df_res.at[file, metric + "_val"] = df_metrics.at[metric, "val"]
        df_res.at[file, metric + "_trn"] = df_metrics.at[metric, "trn"]
        df_res.at[file, metric + "_tst"] = df_metrics.at[metric, "tst"]
        df_res.at[file, metric + "_trn_val"] = df_metrics.at[metric, "trn_val"]
        df_res.at[file, metric + "_val_tst"] = df_metrics.at[metric, "val_tst"]
        df_res.at[file, metric + "_trn_val_tst"] = df_metrics.at[metric, "trn_val_tst"]

    # Params
    cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
    for param_pair in cfg:
        param, val = param_pair.split('=')
        df_res.at[file, param] = val

df_res.set_index('index', inplace=True)
df_res["train_more_val"] = False
df_res["selected"] = False
df_res.loc[df_res["mean_absolute_error_trn"] > df_res["mean_absolute_error_val"], "train_more_val"] = True

first_columns = {
    'selected': 'selected',
    'train_more_val': 'train_more_val',
    'mean_absolute_error_trn': 'MAE trn',
    'mean_absolute_error_val': 'MAE val',
    'mean_absolute_error_tst': 'MAE tst',
    'mean_absolute_error_val_tst': 'MAE val_tst',
    'mean_absolute_error_trn_val_tst': 'MAE trn_val_tst',
    'pearson_corr_coef_trn': 'Pcorr trn',
    'pearson_corr_coef_val': 'Pcorr val',
    'pearson_corr_coef_tst': 'Pcorr tst',
    'pearson_corr_coef_val_tst': 'Pcorr val_tst',
    'pearson_corr_coef_trn_val_tst': 'Pcorr trn_val_tst',
    'mean_absolute_error_cv_mean_trn': 'MAE cv_mean_trn',
    'mean_absolute_error_cv_std_trn': 'MAE cv_std_trn',
    'mean_absolute_error_cv_mean_val': 'MAE cv_mean_val',
    'mean_absolute_error_cv_std_val': 'MAE cv_std_val',
    'pearson_corr_coef_cv_mean_trn': 'Pcorr cv_mean_trn',
    'pearson_corr_coef_cv_std_trn': 'Pcorr cv_std_trn',
    'pearson_corr_coef_cv_mean_val': 'Pcorr cv_mean_val',
    'pearson_corr_coef_cv_std_val': 'Pcorr cv_std_val',
}
df_res = df_res[list(first_columns.keys()) + [col for col in df_res.columns if col not in first_columns]]
df_res.rename(columns=first_columns, inplace=True)
df_res.to_excel(f"{path_runs}/summary.xlsx", index=True, index_label="file")